In [127]:
import os

In [128]:
%pwd

'E:\\parth\\DonorsChoose-Application-Screening-app'

In [129]:
Project_path = "E:\parth\DonorsChoose-Application-Screening-app"
os.chdir(Project_path)

In [130]:
%pwd

'E:\\parth\\DonorsChoose-Application-Screening-app'

In [131]:

"""config/config.yaml"""

# artifacts_root: artifacts

# data_ingestion:
#   root_dir: artifacts/data_ingestion
#   source_URL: https://drive.google.com/file/d/1-4mgFvbEEvCuIar7eas29F5k8hfbRFn3/view?usp=sharing
#   local_data_file: artifacts/data_ingestion/dataset.zip
#   unzip_dir: artifacts/data_ingestion

# data_cleaning:
#   root_dir: artifacts/data_cleaning
#   local_data_trainfile: artifacts/data_ingestion/dataset/train1000.csv
#   local_data_resourcefile: artifacts/data_ingestion/dataset/resource1000.csv


'config/config.yaml'

In [132]:
"""src/entity/config_entity -  added entity for data clean """

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataCleanConfig:
    root_dir: Path
    local_data_trainfile: Path
    local_data_resourcefile: Path

In [133]:
"""data config manager it will create config object for data clean """

from donorschoose.constants import *
from donorschoose.utils.common import read_yaml, create_directories
# from donorschoose.entity.config_entity import DataCleanConfig
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_clean_config(self) -> DataCleanConfig:
        config = self.config.data_cleaning
        
        create_directories([config.root_dir])

        data_clean_config = DataCleanConfig(
            root_dir=config.root_dir,
            local_data_trainfile=config.local_data_trainfile,
            local_data_resourcefile=config.local_data_resourcefile
        )
        return data_clean_config

In [35]:
"""common function for data rread"""
import os
from box.exceptions import BoxValueError
import yaml
from donorschoose import logger
import json
import joblib
from ensure import ensure_annotations
from box import ConfigBox
from pathlib import Path
from typing import Any
import pandas as pd

@ensure_annotations
def read_csv(path_to_csv: Path) -> pd.core.frame.DataFrame:
    """reads csv file and returns 

    Args:
        path_to_csv (str): path like input

    Raises:
        e: empty file

    Returns:
        pd.core.frame.DataFrame
    """
    try:
        with open(path_to_csv) as csv_file:
            dataframe = pd.read_csv(csv_file)
            logger.info(f"csv file: {path_to_csv}, df Shape:{dataframe.shape} loaded successfully")
            return dataframe
    except Exception as e:
        raise e

In [134]:
import pandas as pd
from pathlib import Path

csv_path = Path(Project_path + r"\artifacts\data_ingestion\dataset\train1000.csv")
df = read_csv(csv_path)
print(df.shape)

[2024-02-09 00:24:48,890: INFO: 2445554870: csv file: E:\parth\DonorsChoose-Application-Screening-app\artifacts\data_ingestion\dataset\train1000.csv, df Shape:(1000, 17) loaded successfully]
(1000, 17)


In [159]:
"""component"""

import os
from donorschoose import logger
import re
from tqdm import tqdm

class DataCleaning:
    def __init__(self, config: DataCleanConfig):
        self.config = config
        self.stopwords= ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"]

    def read_files(self) -> pd.core.frame.DataFrame:
        '''
        Fetches data from the specified URLs and returns DataFrames.
        '''
        try: 
            root_dir = self.config.root_dir
            os.makedirs(root_dir, exist_ok=True)

            train_path = self.config.local_data_trainfile
            resource_path = self.config.local_data_resourcefile
            train_df = read_csv(train_path)
            resource_df = read_csv(resource_path)

        except Exception as e:
            raise e

        return train_df, resource_df
    
    def clean_text_column(df_column :pd.core.frame.DataFrame , columnName : str  )-> pd.core.frame.DataFrame :
        """
        this function will remove Spaces and special char from text column
        
        if we have any Nan value then it will fill nan value with 
        whatever have max count category
        """
        df_column =df_column.str.replace(' ','_')
        df_column =df_column.str.replace(',','_')
        df_column =df_column.str.replace('.','')
        df_column =df_column.str.replace('-','_')
        df_column =df_column.str.replace('&','_')
        df_column =df_column.str.replace(' The ','')
        df_column =df_column.str.lower()
        df_column =df_column.str.replace('___','_')
        df_column =df_column.str.replace('__','_')
        
        max_count_id = df_column.value_counts().idxmax()
        df_column=df_column.fillna(max_count_id)
        
        logger.info(f"preprocessed column :{columnName} , unique categories Count : {len(df_column.value_counts())}")

        return df_column

    def decontracted(phrase: str) -> str:
        # specific
        phrase = re.sub(r"won't", "will not", phrase)
        phrase = re.sub(r"can\'t", "can not", phrase)

        # general
        phrase = re.sub(r"n\'t", " not", phrase)
        phrase = re.sub(r"\'re", " are", phrase)
        phrase = re.sub(r"\'s", " is", phrase)
        phrase = re.sub(r"\'d", " would", phrase)
        phrase = re.sub(r"\'ll", " will", phrase)
        phrase = re.sub(r"\'t", " not", phrase)
        phrase = re.sub(r"\'ve", " have", phrase)
        phrase = re.sub(r"\'m", " am", phrase)
        
        return phrase

    def preprocess_text(text_data: list)-> list:
        preprocessed_text = []
        for sentance in tqdm(text_data):
            sent = decontracted(sentance)
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\n', ' ')
            sent = sent.replace('\\"', ' ')
            sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
            sent = ' '.join(e for e in sent.split() if e.lower() not in stopwords)
            preprocessed_text.append(sent.lower().strip())
        return preprocessed_text
    

ModuleNotFoundError: No module named 'Tuple'

In [153]:
project_csv_path = Path(Project_path + r"\artifacts\data_ingestion\dataset\train1000.csv")
resource_csv_path = Path(Project_path + r"\artifacts\data_ingestion\dataset\resource1000.csv")
project_data = read_csv(csv_path)
resource_data =read_csv(resource_csv_path)

[2024-02-09 00:35:42,529: INFO: 2445554870: csv file: E:\parth\DonorsChoose-Application-Screening-app\artifacts\data_ingestion\dataset\train1000.csv, df Shape:(1000, 17) loaded successfully]
[2024-02-09 00:35:42,617: INFO: 2445554870: csv file: E:\parth\DonorsChoose-Application-Screening-app\artifacts\data_ingestion\dataset\resource1000.csv, df Shape:(1000, 17) loaded successfully]


In [155]:
project_data['project_grade_category'] = clean_text_column(project_data['project_subject_categories'] ,'project_subject_categories')
project_data['project_subject_categories'] = clean_text_column(project_data['project_subject_subcategories'] ,'project_subject_subcategories')
project_data['teacher_prefix'] = clean_text_column(project_data['teacher_prefix'] ,'teacher_prefix')
project_data['school_state'] = clean_text_column(project_data['school_state'] ,'school_state')

[2024-02-09 00:35:45,015: INFO: 3332820451: preprocessed column :project_subject_categories , unique category count: 38]
[2024-02-09 00:35:45,097: INFO: 3332820451: preprocessed column :project_subject_subcategories , unique category count: 143]


[2024-02-09 00:35:45,145: INFO: 3332820451: preprocessed column :teacher_prefix , unique category count: 4]
[2024-02-09 00:35:45,189: INFO: 3332820451: preprocessed column :school_state , unique category count: 48]


In [ ]:
#project_subject_categories
#project_subject_subcategories
#teacher_prefix
#school_state

In [156]:
project_data['project_title'] = preprocess_text(project_data['project_title'].values)

100%|██████████| 1000/1000 [00:00<00:00, 1403.68it/s]


In [157]:
project_data["essay"] = project_data["project_essay_1"].map(str) +\
                        project_data["project_essay_2"].map(str) + \
                        project_data["project_essay_3"].map(str) + \
                        project_data["project_essay_4"].map(str)

In [158]:
project_data['essay']= preprocess_text(project_data['essay'].values)

100%|██████████| 1000/1000 [00:05<00:00, 185.53it/s]


In [ ]:
price_data = resource_data.groupby('id').agg({'price':'sum', 'quantity':'sum'}).reset_index()
project_data = pd.merge(project_data, price_data, on='id', how='left')

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(project_data['price'].values.reshape(-1, 1))
project_data['price']=scaler.transform(project_data['price'].values.reshape(-1, 1) )

In [ ]:
project_data = project_data.drop(["project_essay_1",
                                  "project_essay_2",
                                  "project_essay_3",
                                  "project_essay_4",
                                  "teacher_id",
                                  "Unnamed: 0",
                                  "project_submitted_datetime",
                                  "id"], axis=1)



project_data = project_data.rename(columns={'project_subject_categories': 'clean_categories',
                        'project_subject_subcategories': 'clean_subcategories'})
